# UCI : Online Retail
Le but de ce notebook est de pouvoir regrouper un ensemble homogène de comportements clients à travers l'historique des factures abouties et annulées de plusieurs clients 

In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction import DictVectorizer
from ggplot import *

## Chargement des données

In [2]:
PATH= 'xxx'
file_name = 'Online Retail.xlsx'
sheet_name ='Online Retail'

# Chargement du fichier des transactions
data = pandas.read_excel(PATH+file_name,sheet_name=sheet_name)

# Affichage de certaines statistiques descriptives
print(data.shape)
print(data.dtypes)
data.describe()
print(np.unique(data['Country']))

(541909, 8)
InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object
['Australia' 'Austria' 'Bahrain' 'Belgium' 'Brazil' 'Canada'
 'Channel Islands' 'Cyprus' 'Czech Republic' 'Denmark' 'EIRE'
 'European Community' 'Finland' 'France' 'Germany' 'Greece' 'Hong Kong'
 'Iceland' 'Israel' 'Italy' 'Japan' 'Lebanon' 'Lithuania' 'Malta'
 'Netherlands' 'Norway' 'Poland' 'Portugal' 'RSA' 'Saudi Arabia'
 'Singapore' 'Spain' 'Sweden' 'Switzerland' 'USA' 'United Arab Emirates'
 'United Kingdom' 'Unspecified']


### Ajout de variables et restructuration du dataset

In [3]:
data['Country']=data['Country'].astype('category')

 * Vérification du nombre de données manquantes

In [4]:
print(data.isnull().sum(axis=0))

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


* Variable supplémentaire : Calcul du prix 

In [5]:
data['FullPrice'] = data['Quantity']* data['UnitPrice']

* Extraction du mois à partir de la date de la facture

In [6]:
data['Month']= data['InvoiceDate'].dt.month
data['Month'] = data['Month'].astype(str)

* Détection des lignes des transactions annulées

In [7]:
data['TransactionCancelled'] = data['InvoiceNo'].str.startswith('C',False) 

On se restreint aux transactions dont le CustomerID est renseigné. 

In [8]:
data = data[pandas.notnull(data['CustomerID'])]
data=data.reset_index()

Vérification des données manquantes après avoir retiré les transactions dont le CustomerID était null

In [9]:
print(data.isnull().sum(axis=0))

index                   0
InvoiceNo               0
StockCode               0
Description             0
Quantity                0
InvoiceDate             0
UnitPrice               0
CustomerID              0
Country                 0
FullPrice               0
Month                   0
TransactionCancelled    0
dtype: int64


Transformation de la colonne catégorielle 'Country' en plusieurs colonnes, la rendant sparse

In [10]:
dv =DictVectorizer()
z=dv.fit_transform(pandas.DataFrame(data['Country']).to_dict(orient='records'))
countries_dimmed = pandas.DataFrame(z.toarray(),columns=dv.get_feature_names())


Transformation de la colonne catégorielle 'Month' en plusieurs colonnes, la rendant sparse

In [11]:
dv2 =DictVectorizer()
z=dv2.fit_transform(pandas.DataFrame(data['Month']).to_dict(orient='records'))
months_dimmed = pandas.DataFrame(z.toarray(),columns=dv2.get_feature_names())


Fusion des 2 nouvelles matrices sparses avec le reste du dataset

In [12]:
data_final = pandas.concat([data,countries_dimmed,months_dimmed],axis=1)

Ce sont les comportements clients qui nous intéressent, donc on va regrouper le dataset par client avec des variables concernant plutôt des clients que des transactions

In [13]:
data_cust_id_2 = data_final.groupby('CustomerID')

In [14]:
countries_kept = ['Country=Australia', 'Country=Austria',
       'Country=Bahrain', 'Country=Belgium', 'Country=Brazil',
       'Country=Canada', 'Country=Channel Islands', 'Country=Cyprus',
       'Country=Czech Republic', 'Country=Denmark', 'Country=EIRE',
       'Country=European Community', 'Country=Finland', 'Country=France',
       'Country=Germany', 'Country=Greece', 'Country=Iceland',
       'Country=Israel', 'Country=Italy', 'Country=Japan', 'Country=Lebanon',
       'Country=Lithuania', 'Country=Malta', 'Country=Netherlands',
       'Country=Norway', 'Country=Poland', 'Country=Portugal', 'Country=RSA',
       'Country=Saudi Arabia', 'Country=Singapore', 'Country=Spain',
       'Country=Sweden', 'Country=Switzerland', 'Country=USA',
       'Country=United Arab Emirates', 'Country=United Kingdom']

Nombre de clients ayant commandé dans plus d'un pays'

In [15]:
len(data_cust_id_2.Country.apply(np.unique).apply(len).to_frame() >1)

4372

Calculer le nombre de deals par pays par clientID

In [16]:
sum_deals_country_per_customer = data_cust_id_2[countries_kept].apply(sum)

Nombre de transactions annulées par client

In [17]:
cancelled_transac_customer = data_cust_id_2.TransactionCancelled.apply(sum).astype('int').to_frame('SumCancelledTransactions')

Nombre de transactions par client

In [18]:
number_invoices_per_customer = data_cust_id_2.InvoiceNo.apply(set).apply(len).to_frame()

Restructuration du dataset afin de mettre pour chaque client connu le nombre de produits achetés.
Chaque colonne représente un produit, la ligne un client et la valeur représente la somme des quantités de produits achetés

In [28]:
matrix_descr = data_final.pivot_table(index=['CustomerID'], columns=['Description'], values='Quantity',aggfunc='sum')
matrix_descr = matrix_descr.fillna(0).reset_index()

Restructuration du dataset afin de mettre pour chaque client connu le prix total pour un mois donné.
Chaque colonne représente un mois, la ligne un client et la valeur représente la somme des prix

In [29]:
matrix_month = data_final.pivot_table(index=['CustomerID'], columns=['Month'], values='FullPrice',aggfunc='sum')
matrix_month = matrix_month.fillna(0).reset_index()

Fusion des 2 matrices 

In [30]:
matrix = matrix_descr.merge(matrix_month,on='CustomerID',how='inner')

Ajout des colonnes suivantes :
* nombre de transactions par client
* nombre de transactions par pays (une colonne par pays) par client
* nombre de transactions annulées par client

In [40]:
X = matrix.join(number_invoices_per_customer,on='CustomerID',how='inner')
X = X.join(sum_deals_country_per_customer,on='CustomerID',how='inner')
X = X.join(cancelled_transac_customer,on='CustomerID',how='inner')

# Application du modèle de clustering (K-means)
Etant donné que nous sommes en mode non-supervisé, un algorithme de clustering sera utilisé pour regrouper les comportements d'achats clients.

In [42]:
cluster = KMeans(n_clusters=10)
X['no_cluster']= cluster.fit_predict(X[X.columns[2:]])

In [43]:
print('Nombre de clients dans chaque cluster :')
X['no_cluster'].value_counts()

Nombre de clients dans chaque cluster :


0    4233
4     119
9       9
1       5
7       1
3       1
6       1
2       1
5       1
8       1
Name: no_cluster, dtype: int64

Affichage des clusters sur les 2 axes avec les 2 plus grandes variances expliquées

In [33]:
pca = PCA(n_components=2)
X['x'] = pca.fit_transform(X[X.columns[1:]])[:,0]
X['y'] = pca.fit_transform(X[X.columns[1:]])[:,1]

In [34]:
customer_clusters = X[['CustomerID', 'no_cluster', 'x', 'y']]
customer_clusters.head()

,CustomerID,no_cluster,x,y
0,12346,0,-562.214381,-32.988734
1,12347,6,667.981764,958.973985
2,12348,0,-63.599147,-13.241046
3,12349,0,14.701642,-296.574769
4,12350,0,-509.514089,41.839902


Affichage des points sur les 2 axes 

In [36]:
ggplot(X, aes(x='x', y='y', color='no_cluster')) + \
    geom_point(size=75) + \
    ggtitle("Customers Grouped by Cluster")

c:\users\wymeka\documents\python\python\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


<ggplot: (-9223372036836878539)>

### Annexes :
Il était aussi possible de diminuer le nombre de produits en les regroupant. Ce regroupement pourrait être effectué en calculant la fréquence de chaque mot de la description du produit et utiliser un algorithme de clustering (ici par exemple un NMF) pour regrouper les produits.

In [ ]:
from sklearn.feature_extraction.text import  TfidfVectorizer
tf_vectorizer = TfidfVectorizer(analyzer='word', min_df = 0)
tf = tf_vectorizer.fit_transform(data['Description'])

In [ ]:
from sklearn.decomposition import NMF
cluster_products = NMF(init='nndsvd', random_state=1,n_components=10)
a=pandas.DataFrame(cluster_products.fit_transform(tf))
b=a.idxmax(axis=1)

In [ ]:
products_clustered = pandas.concat([data['Description'],b],axis=1)